<a href="https://colab.research.google.com/github/newb-dev-1008/Human-PokeDex/blob/master/Slow%20Fusion%20-%20(Distributed%20Training%20-%202nd).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Tests to try
# 1. Change in number of images per box (16)
# 2. Kernel sizes
# 3. Addition of Residual blocks
# 4. Change in number of images per layer (from 2 to 4) in the temporal dimension
# 5. Check the code for prediction in x_test
# 6. Check out the graphs per epoch you want to plot

# **Number of images: 16**
With number of images per layer: 6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.models import Model
from tensorflow.keras import utils as np_utils
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import argparse
import pickle
import cv2
import os
from sklearn.utils import shuffle
import random
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Check available TensorFlow devices
from tensorflow import config
config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
inputData = "/content/drive/MyDrive/Datasets/Annotated"
arrPaths = []
arrLbls = []
lbls = []
temporal_dimension = 10
labelNumber = {
    'Abuse' : 0,
    'Assault' : 1,
    'Fighting' : 2,
    'Normal' : 3,
    'Robbery' : 4,
    'Vandalism' : 5
}

for labelFolder in os.listdir(inputData):
    labelFolderPath = (inputData + "/" + labelFolder)
    
    for clip in os.listdir(labelFolderPath):
        count = 0
        clipPath = (inputData + "/" + labelFolder + "/" + clip)
        
        for frame in range(count, len(os.listdir(clipPath)), temporal_dimension):
            if ((count + 10) < len(os.listdir(clipPath))):
                tempArr = os.listdir((inputData + "/" + labelFolder + "/" + clip))[count : (count + temporal_dimension)]
                for i in range(len(tempArr)):
                    tempArr[i] = inputData + "/" + labelFolder + "/" + clip + "/" + tempArr[i]
                arrPaths.append(tempArr)
                del tempArr
                lbls.append(labelNumber[labelFolder])
                count += temporal_dimension

In [ ]:
# arrPaths and lbls are the lists of interest
# arrPaths = [[imagePath1, imagePath2, ..., imagePath10], [imagePath1, imagePath2, ..., imagePath10] ... for all clips in video]
# lbls = [label1, label2, label3, ... for each 10-frame clip in video]

[x_train, x_test, y_train, y_test] = train_test_split(arrPaths, lbls, test_size = 0.25, shuffle = True, stratify = None)

In [ ]:
# Create a generator for yielding batchSize number of 3D bundles (consisting of 10 images each)
# Attach generator for train and test
# Train 3DCNN with the train generator

dataTrainPaths = [[x_train[i], y_train[i]] for i in range(len(y_train))]
dataTestPaths = [[x_test[i], y_test[i]] for i in range(len(y_test))]

In [ ]:
# Check if all arrPath elements have exactly 'temporal_dimension' image paths 

flag = 0
temp = []
for i in range(len(arrPaths)):
    if (len(arrPaths[i]) != temporal_dimension):
        flag = 1
        temp.append(i)
        # break

if (flag == 1):
    print("Not balanced.")
    print(temp)
    print(len(temp))
else:
    print("Perfect.")

Perfect.


In [ ]:
# Check if all x_train elements have exactly 'temporal_dimension' image paths 

# For DataTrainPaths
flag = 0
temp = 0
for i in range(len(dataTrainPaths)):
    if (len(dataTrainPaths[i][0]) != temporal_dimension):
        flag = 1
        temp = i
        break

if (flag == 1):
    print("DataTrainPaths not balanced.")
    print(dataTrainPaths[temp][0])
else:
    print("DataTrainPaths Perfect.")

# For DataTestPaths
flag = 0
temp = 0
for i in range(len(dataTestPaths)):
    if (len(dataTestPaths[i][0]) != temporal_dimension):
        flag = 1
        temp = i
        break

if (flag == 1):
    print("DataTestPaths not balanced.")
    print(dataTrainPaths[temp][0])
else:
    print("DataTestPaths Perfect.")

DataTrainPaths Perfect.
DataTestPaths Perfect.


In [ ]:
def data_generator(dataPaths, batch_size, sfl = True):              
    """
    Yields the next training batch.
    data is an array [[[frame1_filename,frame2_filename,…frame16_filename],label1], [[frame1_filename,frame2_filename,…frame16_filename],label2],……….].
    """
    LABELS = ["Abuse", "Assault", "Fighting", "Normal", "Robbery", "Vandalism"]
    num_samples = len(dataPaths)
    
    if sfl:
        dataPaths = shuffle(dataPaths, random_state = 2)
        
    while True:   
        for offset in range(0, num_samples, batch_size):
            # print('Starting index: ', offset) 
            
            # Get the samples you'll use in this batch
            batch_samples = dataPaths[offset : (offset + batch_size)]
            
            # Initialise X_train and y_train arrays for this batch
            X_train = []
            Y_train = []
            
            # For each sample
            for batch_sample in batch_samples:
                # Load image paths (X)
                x = batch_sample[0]
                
                # Read label (y)
                y = batch_sample[1]
                
                temp_data_list = []
                for img in x:
                    try:
                        # print(img.split('\\')[-1], "\n")
                        img = cv2.imread(img)
                        img = cv2.resize(img, (224, 224))
                        temp_data_list.append(img)
                    except Exception as e:
                        print(e)
                        print('Error reading file: ', img)                      
                
                # Add sample to arrays
                X_train.append(np.array(temp_data_list))
                Y_train.append(y)
    
            # Make sure they're numpy arrays 
            X_train = np.array(X_train)
            
            #X_train = np.rollaxis(X_train,1,4)
            Y_train = np.array(Y_train)
            
            # create one hot encoding for training in keras
            Y_train = np_utils.to_categorical(Y_train, len(LABELS))
    
            # yield the next training batch            
            yield X_train, Y_train

In [ ]:
train_generator = data_generator(dataTrainPaths, batch_size = 2)
test_generator = data_generator(dataTestPaths, batch_size = 2)

timepass_x, timepass_y = next(train_generator)

print("Shape of X: ", timepass_x.shape)
print("Shape of Y: ", timepass_y.shape)

Shape of X:  (2, 10, 224, 224, 3)
Shape of Y:  (2, 6)


In [ ]:
x_0 = timepass_x[0]
y_0 = timepass_y[0]

print("Shape of X_0: ", x_0.shape)
print("Shape of Y_0: ", y_0.shape)

Shape of X_0:  (10, 224, 224, 3)
Shape of Y_0:  (6,)


In [ ]:
# Plotting the images in one 3D block (of 10 images)
num_of_images = temporal_dimension
LABELS = ["Abuse", "Assault", "Fighting", "Normal", "Robbery", "Vandalism"]

# activity = LABELS[y_0.index(1)]
fig = plt.figure(figsize = (8, 8)) 

# plt.title("One sample with {} frames | Activity: {}".format(num_of_images, activity))
plt.title("One sample with {} frames".format(num_of_images))
subplot_num = int(np.ceil(np.sqrt(num_of_images)))
for i in range(int(num_of_images)):
    ax = fig.add_subplot(subplot_num, subplot_num, i+1)
    #ax.imshow(output_image[0,:,:,i],interpolation='nearest' ) #to see the first filter
    ax.imshow(x_0[i,:,:,::-1])
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
plt.show()
plt.savefig('/content/drive/MyDrive/images_slowfusion-2nd.png')

In [ ]:
print(set(y_train))

{0, 1, 2, 3, 4, 5}


In [ ]:
LABELS = ["Abuse", "Assault", "Fighting", "Normal", "Robbery", "Vandalism"]
testing = np_utils.to_categorical(y_train, len(LABELS))
print(testing)

[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [ ]:
# Building and training the model
from tensorflow.keras.layers import AveragePooling3D
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv3D(32, kernel_size = (3, 3, 4), strides = (1, 1, 2), input_shape = (temporal_dimension, 224, 224, 3), padding = 'same'))
model.add(BatchNormalization(axis = 2))
model.add(AveragePooling3D(pool_size = (2, 2, 2), strides = None, padding = 'same'))
model.add(Conv3D(16, kernel_size = (3, 3, 2), strides = (1, 1, 2), padding = 'same'))
model.add(BatchNormalization(axis = 2))
model.add(AveragePooling3D(pool_size = (2, 2, 2), strides = None, padding = 'same'))
model.add(Conv3D(16, kernel_size = (3, 3, 2), padding = 'same'))
model.add(BatchNormalization(axis = 2))
model.add(AveragePooling3D(pool_size = (2, 2, 2), strides = None, padding = 'same'))

# FC Layers
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(len(LABELS), activation = 'softmax'))

'''
model.add(Conv3D(32, kernel_size = (3, 3, 3), input_shape = (10, 224, 224, 3), padding = 'same'))
model
model.add(Activation('relu'))
model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(Dropout(0.25))
model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation("softmax"))
'''

model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(), metrics=[tf.keras.metrics.Accuracy(),
                                                                                 tf.keras.metrics.CategoricalAccuracy(),
                                                                                 tf.keras.metrics.TopKCategoricalAccuracy(),
                                                                                 tf.keras.metrics.Precision(),
                                                                                 tf.keras.metrics.Recall(), 
                                                                                 tf.keras.metrics.TruePositives(), 
                                                                                 tf.keras.metrics.TrueNegatives(), 
                                                                                 tf.keras.metrics.FalsePositives(), 
                                                                                 tf.keras.metrics.FalseNegatives()])

#plot_model(model, show_shapes = True, to_file = 'model.png')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 10, 224, 112, 32)  3488      
_________________________________________________________________
batch_normalization (BatchNo (None, 10, 224, 112, 32)  896       
_________________________________________________________________
average_pooling3d (AveragePo (None, 5, 112, 56, 32)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 5, 112, 28, 16)    9232      
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 112, 28, 16)    448       
_________________________________________________________________
average_pooling3d_1 (Average (None, 3, 56, 14, 16)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 3, 56, 14, 16)     4

In [ ]:
# Saving checkpoints while training

checkpointsPath = '/content/drive/MyDrive/Checkpoints/2nd Colab Checkpoint/checkpoint.ckpt'

checkpointCallback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpointsPath, 
                                                        save_best_only = False, 
                                                        save_weights_only = True, 
                                                        verbose = 1)
tensorboardCallback = tf.keras.callbacks.TensorBoard(log_dir = "/content/drive/MyDrive/Checkpoints/2nd Colab Checkpoint/TensorBoard Logs",
                                                    write_graph = True,
                                                    update_freq = 'epoch')

In [ ]:
# Training the model
n_epochs = 50

with tf.device('/gpu:0'):
    H = model.fit(train_generator, 
                  steps_per_epoch = len(dataTrainPaths), 
                  epochs = n_epochs, 
                  validation_data = test_generator, 
                  validation_steps = len(dataTestPaths), 
                  callbacks = [checkpointCallback, tensorboardCallback])
    
# Serialize the model to disk
print("Serializing network...")
modelPath = '/content/drive/MyDrive/Saved Models/2nd Colab Model/slowfusion_model'

# Model can be saved in h5 or .savedmodel
model.save(modelPath, save_format="h5")
print("Model serialized and saved.\n")


# serialize the label binarizer to disk
# lbPath = r'C:\Users\Yash Umale\Documents\6th Sem\Open Lab\Python Files\Crime Detection\Models\lb.pickle'
# f = open(lbPath, "wb")
# f.write(pickle.dumps(lb))
# f.close()

Epoch 1/50
  15/3894 [..............................] - ETA: 14:29:53 - loss: 7.1873 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3667 - top_k_categorical_accuracy: 0.8000 - precision: 0.3929 - recall: 0.3667 - true_positives: 11.0000 - true_negatives: 133.0000 - false_positives: 17.0000 - false_negatives: 19.0000

KeyboardInterrupt: ignored

In [ ]:
# Saving the history object of trained model
import json
history_dict = H.history

# Save it under the form of a json file
JSON_path = '/content/drive/MyDrive/Checkpoints/2nd Colab Checkpoint/History/history.json'
json.dump(history_dict, open(JSON_path, 'w'))

In [ ]:
# Plot the training loss and accuracy

N = n_epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plotPath = '/content/drive/MyDrive/Plots/2nd Colab Plot/slowfusion_trend.png'
plt.savefig(plotPath)

In [ ]:
# Invoke the saved model for evaluation
from tensorflow.keras.models import load_model

savedModelPath = '/content/drive/MyDrive/Saved Models/2nd Colab Model/slowfusion_model'
savedModel = load_model(savedModelPath)
savedModel.summary()

In [ ]:
# Evaluate the network

print("Evaluating network...")
predictions = savedModel.predict(test_generator)
print(classification_report(y_test.argmax(axis = 1), predictions.argmax(axis = 1), target_names = LABELS))